# SQUANCH demonstration: quantum error correction with the Shor code

In [1]:
from squanch import *
from scipy.stats import unitary_group
import copy
import numpy as np
import matplotlib.image as image
import matplotlib.pyplot as plt
%matplotlib inline

This Jupyter notebook contains the source code for the [Shor code demonstration](https://att-innovate.github.io/squanch/demos/quantum-error-correction.html). Refer to the documentation for more detailed explanations. 

In this demonstration, we show how to use SQUANCH's channel and error modules to simulate quantum errors in a transmitted message, which we correct for using the [Shor code](https://en.wikipedia.org/wiki/Quantum_error_correction#The_Shor_code). This error correction model encodes a single logical qubit into the product of 9 physical qubits and is capable of correcting for arbitrary single-qubit errors. A circuit diagram of the protocol we will implement is shown below, where $E$ represents a quantum channel which can arbitrarily corrupt a single qubit:

![](https://upload.wikimedia.org/wikipedia/commons/a/a9/Shore_code.svg)

## Agent logic

In [2]:
class Alice(Agent):
    '''Alice sends an arbitrary Shor-encoded state to Bob'''
    def shor_encode(self, qsys):
        # psi is state to send, q1...q8 are ancillas from top to bottom in diagram
        psi, q1, q2, q3, q4, q5, q6, q7, q8 = qsys.qubits
        # Gates are enumerated left to right, top to bottom from figure
        CNOT(psi, q3)
        CNOT(psi, q6)
        H(psi)
        H(q3)
        H(q6)
        CNOT(psi, q1)
        CNOT(psi, q2) 
        CNOT(q3, q4)
        CNOT(q3, q5)
        CNOT(q6, q7)
        CNOT(q6, q8)
        return psi, q1, q2, q3, q4, q5, q6, q7, q8
    
    def run(self):
        for qsys in self.stream:
            # send the encoded qubits to Bob 
            for qubit in self.shor_encode(qsys):
                self.qsend(bob, qubit)
                
class DumbAlice(Agent):
    '''DumbAlice sends a state to Bob but forgets to error-correct!'''   
    def run(self):
        for qsys in self.stream:
            for qubit in qsys.qubits:
                self.qsend(dumb_bob, qubit)

In [3]:
class Bob(Agent):
    '''Bob receives Alice's qubits and applied error correction'''
    def shor_decode(self, psi, q1, q2, q3, q4, q5, q6, q7, q8):
        # same enumeration as Alice
        CNOT(psi, q1)
        CNOT(psi, q2)
        TOFFOLI(q2, q1, psi)
        CNOT(q3, q4)
        CNOT(q3, q5)
        TOFFOLI(q5, q4, q3)
        CNOT(q6, q7)
        CNOT(q6, q8)
        TOFFOLI(q7, q8, q6) # Toffoli control qubit order doesn't matter
        H(psi)
        H(q3)
        H(q6)
        CNOT(psi, q3)
        CNOT(psi, q6)
        TOFFOLI(q6, q3, psi)
        return psi # psi is now the original state
    
    def run(self):
        measurement_results = []
        for _ in self.stream:
            # Bob receives 9 qubits representing Alice's encoded state
            received = [self.qrecv(alice) for _ in range(9)]
#             for i in range(9):
#                 q = self.qrecv(alice)
#                 received.append(q)
            # Decode and measure the original state
            psi_true = self.shor_decode(*received)
            measurement_results.append(psi_true.measure())
        self.output(measurement_results)
        
class DumbBob(Agent):
    '''DumbBob receives a state from Alice but does not error-correct'''
    def run(self):
        measurement_results = []
        for _ in self.stream:
            received = []
            for i in range(9):
                received.append(self.qrecv(dumb_alice))
            psi_true = received[0]
            measurement_results.append(psi_true.measure())
        self.output(measurement_results)

## Quantum error model

In [4]:
# class ShorError(QError):
#     '''Randomly apply a bit flip, a sign flip, or both to any given transmitted qubit'''

#     def __init__(self, qchannel):
#         '''
#         Instatiate the error model from the parent class
#         :param QChannel qchannel: parent quantum channel
#         '''
#         QError.__init__(self, qchannel)
#         self.count = 0
#         self.error_applied = False

#     def apply(self, qubit):
#         '''
#         Randomly apply a bit flip, sign flip, or both to the qubit
#         :param Qubit qubit: qubit from quantum channel
#         :return: either unchanged qubit or None
#         '''
#         # qubit could be None if combining with other error models, such as attenuation
#         if not self.error_applied and qubit is not None:
#             if np.random.rand() > 0.5: 
#                 X(qubit) # apply bit flip
#                 self.error_applied = True
#             if np.random.rand() > 0.5: 
#                 Z(qubit) # apply sign flip
#                 self.error_applied = True
#         self.count += 1
#         if self.count >= 9:
#             self.count = self.count % 9
#             self.error_applied = False
#         return qubit

In [5]:
class ShorError(QError):

    def __init__(self, qchannel):
        '''
        Instatiate the error model from the parent class
        :param QChannel qchannel: parent quantum channel
        '''
        QError.__init__(self, qchannel)
        self.count = 0 # tracks which qubit out of 9 we're at
        self.error_applied = False # if another error can be applied

    def apply(self, qubit):
        '''
        Apply a random unitary operation to one of the qubits in a set of 9
        :param Qubit qubit: qubit from quantum channel
        :return: either unchanged qubit or None
        '''
        if self.count == 0:
            self.error_applied = False
        # qubit could be None if combining with other error models, such as attenuation
        if not self.error_applied and qubit is not None:
            if np.random.rand() < 0.1: # apply the error
                random_unitary = unitary_group.rvs(2) # pick a random U(2) matrix
                qubit.apply(random_unitary)
                self.error_applied = True
        # update count and reset error if necessary
        self.count = (self.count + 1) % 9
        return qubit

## Channel model

In [6]:
class ShorQChannel(QChannel):
    '''Represents a quantum channel with a Shor error applied'''
    
    def __init__(self, from_agent, to_agent):
        QChannel.__init__(self, from_agent, to_agent)
        # register the error model
        self.errors = [ShorError(self)] 

## Running the simulation

In [7]:
def to_bits(string):
    '''Convert a string to a list of bits'''
    result = []
    for c in string:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return result

def from_bits(bits):
    '''Convert a list of bits to a string'''
    chars = []
    for b in range(int(len(bits) / 8)):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

In [9]:
# Prepare a message to send
msg = "Peter"# Shor once lived in Ruddock 238! But who was Airman?"
bits = to_bits(msg)

# Allocate memory and output
mem = Agent.shared_hilbert_space(9, len(bits)) # 9 qubits per encoded state
out = Agent.shared_output()

# Encode the message as spin eigenstates
stream = QStream.from_array(mem)
for bit, qsystem in zip(bits, stream):
    q = qsystem.qubit(0)
    if bit == 1: 
        X(q) 

# Alice and Bob will use error correction
alice = Alice(mem, out)
bob = Bob(mem, out)
alice.qconnect(bob)#, ShorQChannel)

# Dumb agents won't use error correction
# mem2 = copy.deepcopy(mem)
# dumb_alice = DumbAlice(mem2, out)
# dumb_bob = DumbBob(mem2, out)
# dumb_alice.qconnect(dumb_bob, ShorQChannel)

# Run everything and record results
# Simulation(dumb_alice, dumb_bob, alice, bob).run()
Simulation(alice, bob).run()

# print("DumbAlice sent:   {}".format(msg))
# print("DumbBob received: {}".format(from_bits(out["DumbBob"])))
# print("")
print("Alice sent:       {}".format(msg))
print("Bob received:     {}".format(from_bits(out["Bob"])))



Alice sent:       Peter
Bob received:     Peter


In [56]:
np.random.rand()

0.6144986065374577

In [55]:
mem = Agent.shared_hilbert_space(9, len(bits)) # 9 qubits per encoded state

In [14]:
mem

array([[[ 1.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
        ..., 
        [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j]],

       [[ 1.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
        ..., 
        [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j]],

       [[ 1.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
  

In [22]:
print(bits)
print(out["Bob"])

[0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0]
[1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0]


In [34]:
print(bits)
print(out["Bob"])

[0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0]
[1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0]


In [9]:
print(bits)
print(out["Bob"])

[0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0]
[0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1]


In [34]:
print(bits)
print(out["Bob"])

[0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0]
[0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0]


In [14]:
print(bits)
print(out["Bob"])

[0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0]
[0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0]
